In [ ]:
import sep
from subprocess import Popen, PIPE
import numpy as np
from astropy.io import fits
from importlib import reload
import time
from ics.cobraCharmer import pfi as pfiControl
reload(pfiControl)

In [ ]:
mod1Cobras = pfiControl.PFI.allocateCobraRange(range(1,2))
allCobras = mod1Cobras
oneCobra = pfiControl.PFI.allocateCobraList([(1,2)])
twoCobras = pfiControl.PFI.allocateCobraList([(1,2), (1,5)])

# partition module 1 cobras into non-interfering sets
moduleCobras = {}
for group in 1,2,3:
    cm = range(group,58,3)
    mod = [1]*len(cm)
    moduleCobras[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))
group1Cobras = moduleCobras[1]
group2Cobras = moduleCobras[2]
group3Cobras = moduleCobras[3]

# partition module 1 cobras into odd and even sets
moduleCobras2 = {}
for group in 1,2:
    cm = range(group,58,2)
    mod = [1]*len(cm)
    moduleCobras2[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))
oddCobras = moduleCobras2[1]
evenCobras = moduleCobras2[2]

In [ ]:
pfi = pfiControl.PFI(fpgaHost='128.149.77.24') #'fpga' for real device.
#pfi = pfiControl.PFI(fpgaHost='localhost', doLoadModel=False)
pfi.loadModel('../xml/precise.xml')
pfi.setFreq(allCobras)

In [ ]:
# Calculate outward angles
oddMoves = pfi.thetaToLocal(oddCobras, [np.deg2rad(270)]*len(oddCobras))
oddMoves[oddMoves>1.85*np.pi] = 0

evenMoves = pfi.thetaToLocal(evenCobras, [np.deg2rad(90)]*len(evenCobras))
evenMoves[evenMoves>1.85*np.pi] = 0

allMoves = np.zeros(57)
allMoves[::2] = oddMoves
allMoves[1::2] = evenMoves

def moveCobra(c, theta, phi):
    pfi.moveSteps([allCobras[c-1]], np.zeros(1)+theta, np.zeros(1)+phi)

def moveCobras(cs, theta, phi):
    cobs = []
    for c in cs:
        cobs.append(allCobras[c-1])
    pfi.moveSteps(cobs, np.array(theta), np.array(phi))

In [ ]:
def lazyIdentification(centers, spots, radii=None):
    n = len(centers)
    if radii is not None and len(radii) != n:
        raise RuntimeError("number of centers must match number of radii")
    ans = np.empty(n, dtype=int)
    for i in range(n):
        dist = np.absolute(spots - centers[i])
        j = np.argmin(dist)
        if radii is not None and np.absolute(centers[i] - spots[j]) > radii[i]:
            ans[i] = -1
        else:
            ans[i] = j
    return ans

In [ ]:
# move one cobra
moveCobra(1, -400, 0)

In [ ]:
# Home phi
pfi.moveAllSteps(allCobras, 0, -5000)

In [ ]:
# Home theta
pfi.moveAllSteps(allCobras, -10000, 0)

In [ ]:
# Target convergence test

# go Home first
#pfi.moveAllSteps(allCobras, 0, -5000)
#pfi.moveAllSteps(allCobras, -10000, 0)

# generate target positions, [pi/5, pi] for theta, [pi/6, pi*(1/2)] for phi
mapping = [e for e in range(1,58) if e not in {1, 39, 43, 54}]
nCobras = len(mapping)
myCobras = pfiControl.PFI.allocateCobraList([(1,e) for e in mapping])
thetaTargets = (np.random.random_sample(nCobras)*4+1) * (np.pi/5)
phiTargets = (np.random.random_sample(nCobras)*2+1) * (np.pi/6)
targets = pfi.anglesToPositions(myCobras, thetaTargets, phiTargets)
threshold = 2.0
maxTries = 10

# define two cobra groups for two cameras
group1 = np.array(mapping[:26]) - 1
group2 = np.array(mapping[26:]) - 1

In [ ]:
# Do first big move
pfi.moveXYfromHome(myCobras, targets)

ntries = 0
pos = np.zeros((len(targets), maxTries+1), dtype=complex)
while True:
    # check current positions, first exposing
    p1 = Popen(["/home/pfs/IDSControl/idsexposure", "-d", "1", "-e", "18", "-f", "data/cam1_"], stdout=PIPE)
    p1.communicate()
    p2 = Popen(["/home/pfs/IDSControl/idsexposure", "-d", "2", "-e", "18", "-f", "data/cam2_"], stdout=PIPE)
    p2.communicate()

    # extract sources and do fiber identification
    data1 = fits.getdata('data/cam1_0001.fits').astype(float)
    ext1 = sep.extract(data1, 100)
    idx1 = lazyIdentification(pfi.calibModel.centers[group1], ext1['x'] + ext1['y']*(1j))
    data2 = fits.getdata('data/cam2_0001.fits').astype(float)
    ext2 = sep.extract(data2, 100)
    idx2 = lazyIdentification(pfi.calibModel.centers[group2], ext2['x'] + ext2['y']*(1j))
    curPos = np.concatenate((ext1[idx1]['x'] + ext1[idx1]['y']*(1j), ext2[idx2]['x'] + ext2[idx2]['y']*(1j)))
    pos[:,ntries] = curPos
    print(curPos)

    # check position errors
    done = np.abs(curPos - targets) <= threshold
    if np.all(done):
        print('Convergence sequence done')
        break
    if ntries >= maxTries:
        print(f'Reach max {maxTries} tries, gave up')
        break
    ntries += 1

    # move again
    pfi.moveXY(myCobras, curPos, targets)